In [32]:
import rdflib
from rdflib import Namespace
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, SKOS, OWL, RDF, RDFS, VOID, XMLNS, XSD
import os

In [33]:

train_path = r"workspace\data\YAGO3-10\train.txt"
test_path = r"workspace\data\YAGO3-10\test.txt"
valid_path = r"workspace\data\YAGO3-10\valid.txt"

def read_nodes(lst):
    nodes = set()
    for path in lst:
        content = None
        with open(path, encoding="utf8") as f:
            content = f.readlines()
        content = [x.strip() for x in content]

        for line in content:
            head,rel,tail = line.split("\t")
            nodes.add(head)
            nodes.add(tail)
    return nodes

def read_line(path):
    with open(path, encoding="utf8") as infile:
        c = 0
        while True:
            line = infile.readline()
            if not line:
                break
            if c % 1000 == 0:
                print(c)
            yield line
            c = c + 1

## Reduce graph

In [26]:
nodes = read_nodes([train_path, test_path, valid_path])


outfile = open("workspace\labels\yago310.nt","w", encoding="utf8")

namespace = "http://yago-knowledge.org/resource/"

for line in read_line(r"workspace\labels\yago-wd-labels.nt"):
    s,p,o,_ = line.split("\t")
    if s.replace(namespace, "")[1:-1] in nodes and o.endswith("@en"):
        if "rdf-schema#label" in p or "rdf-schema#comment" in p:
            outfile.write(line)

for line in read_line(r"workspace\labels\yago-wd-full-types.nt"):
    s,p,o,_ = line.split("\t")
    if s.replace(namespace, "")[1:-1] in nodes:
        outfile.write(line)

outfile.close()

2895000
2896000
2897000
2898000
2899000
2900000
2901000
2902000
2903000
2904000
2905000
2906000
2907000
2908000
2909000
2910000
2911000
2912000
2913000
2914000
2915000
2916000
2917000
2918000
2919000
2920000
2921000
2922000
2923000
2924000
2925000
2926000
2927000
2928000
2929000
2930000
2931000
2932000
2933000
2934000
2935000
2936000
2937000
2938000
2939000
2940000
2941000
2942000
2943000
2944000
2945000
2946000
2947000
2948000
2949000
2950000
2951000
2952000
2953000
2954000
2955000
2956000
2957000
2958000
2959000
2960000
2961000
2962000
2963000
2964000
2965000
2966000
2967000
2968000
2969000
2970000
2971000
2972000
2973000
2974000
2975000
2976000
2977000
2978000
2979000
2980000
2981000
2982000
2983000
2984000
2985000
2986000
2987000
2988000
2989000
2990000
2991000
2992000
2993000
2994000
2995000
2996000
2997000
2998000
2999000
3000000
3001000
3002000
3003000
3004000
3005000
3006000
3007000
3008000
3009000
3010000
3011000
3012000
3013000
3014000
3015000
3016000
3017000
3018000
3019000


In [44]:
g = rdflib.Graph()
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.parse("workspace\labels\yago310.nt", format="nt")

<Graph identifier=N48fa7675097f47ad9ab995ba1217acc2 (<class 'rdflib.graph.Graph'>)>

In [49]:
out = rdflib.Graph()
out.bind("rdf", RDF)
out.bind("rdfs", RDFS)
ai = Namespace("http://ai-strategies.org/ns/")
out.bind("ai", ai)

for row in g.subject_objects(RDFS.label):
    entity = row[0]
    label = row[1]


    out.add((
        entity,
        RDFS.label,
        rdflib.Literal(label, datatype=XSD.string)
    ))

    out.add((
        entity,
        ai.wwwresource,
        rdflib.Literal(entity, datatype=XSD.string)
    ))

    description = None
    for description in g.objects(entity, RDFS.comment):
        break
    if description != None:
        out.add((
            entity,
            RDFS.comment,
            rdflib.Literal(description, datatype=XSD.string)
        ))

    entity_type = None
    for entity_type in g.objects(entity, RDF.type):
        entity_type = entity_type.replace("http://schema.org/", "").replace("http://yago-knowledge.org/resource/", "")
        out.add((
            entity,
            RDF.type,
            rdflib.Literal(entity_type, datatype=XSD.string)
        ))




In [50]:
relations = {
    "isPoliticianOf":"is politician of",
    "hasNeighbor":"has neighbor",
    "playsFor":"plays for",
    "isInterestedIn":"is interested in",
    "hasCurrency":"has currency",
    "dealsWith":"deals with",
    "edited":"edited",
    "livesIn":"lives in",
    "owns":"owns",
    "created":"created",
    "hasChild":"has child",
    "influences":"influences",
    "hasWebsite":"has website",
    "hasCapital":"has capital",
    "hasOfficialLanguage":"has official language",
    "worksAt":"works at",
    "wroteMusicFor":"wrote music for",
    "isCitizenOf":"is citizen of",
    "hasWonPrize":"has won prize",
    "actedIn":"acted in",
    "exports":"exports",
    "hasAcademicAdvisor":"has academic advisor",
    "isKnownFor":"is known for",
    "graduatedFrom":"graduated from",
    "isLocatedIn":"is located in",
    "happenedIn":"happened in",
    "directed":"directed",
    "isMarriedTo":"is married to",
    "isConnectedTo":"is connected to",
    "diedIn":"died in",
    "hasGender":"has gender",
    "participatedIn":"participated in",
    "imports":"imports",
    "isAffiliatedTo":"is affiliated to",
    "isLeaderOf":"is leader of",
    "wasBornIn":"was born in",
    "hasMusicalRole":"has musical role",
}

for relation, label in relations.items():
    out.add((
        kg.term(relation),
        RDFS.label,
        rdflib.Literal(label, datatype=XSD.string)
    ))

In [51]:
out.serialize(os.path.abspath(r"workspace/graphs/yago310.ttl"),format="turtle")

In [52]:

outfile = open(r"workspace/graphs/yago310.ttl", 'a', encoding="utf8")

def read_set(path, typ):
    content = None
    with open(path, encoding="utf8") as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    for line in content:
        head,rel,tail = line.split("\t")
        outfile.write(f"<<<http://yago-knowledge.org/resource/{head}> <http://yago-knowledge.org/resource/{rel}> <http://yago-knowledge.org/resource/{tail}>>> ai:split ai:{typ} . " + "\n")

read_set(train_path, 'train')
read_set(test_path, 'test')
read_set(valid_path, 'valid')

outfile.close()